# Neural Style Transfer


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from utils import nst
import cv2
%matplotlib inline

There are two types of costs: content and style

In [0]:
def content_cost(a_C, a_G):
  """
  Computes the content cost

  a_C - tensor representing content of image C
  a_G - tensor representing content of image G
  """

  m, n_H, n_W, n_C = a_G.get_shape().as_list()

  a_C_unr = tf.reshape(a_C, shape=[m, -1, n_C])
  a_G_unr = tf.reshape(a_G, shape=[m, -1, n_C])

  J_content = tf.reduce_sum(tf.square(tf.subtract(a_C_unr, a_G_unr))) / (4 * n_W * n_C * n_H)

  return J_content


In [0]:
def layer_style_coct(a_C, a_G):
  """
  Computes the style cost for one layer

  a_C - tensor representing content of image C
  a_G - tensor representing content of image G

  """
  m, n_H, n_W, n_C = a_G.get_shape().as_list()

  a_C = tf.transpose(tf.reshape(a_C, shape=[-1, n_C]), perm=[1, 0])
  a_G = tf.transpose(tf.reshape(a_G, shape=[-1, n_C]), perm=[1, 0])

  #computing Gram matix
  GC = tf.matmul(a_C, tf.transpose(a_C))
  GG = tf.matmul(a_G, tf.transpose(a_G))

  J_style_for_layer = tf.reduce_sum(tf.square(tf.subtract(GC, GG))) / (4 * (n_C * n_W * n_H)**2)

  return J_style_for_layer



In [0]:
#let's define weight of costs for each conv_layer (with different depth in net)
#for now they represent the same value for all of them
LAYER_WEIGHTS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [0]:
def full_style_cost(model, LAYER_WEIGHTS):
  """
  Computes full style cost between two images
  """
  J_style = 0

  for layer_name, weight in LAYER_WEIGHTS:
    out = model[layer_name]

    a_C = sess.run(out)
    a_G = out

    J_style += weight * layer_style_coct(a_C, a_G)

  return J_style



In [0]:
def total_cost(J_content, J_style, coeff_1 = 10, coeff_2 = 120):
  """
  Computes full cost, weighting between content(coeff_1) and style(coeff_2)
  """

  return coeff_1 * J_content + coeff_2 * J_style

In [0]:
#start interactive session
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [0]:
content_image = cv2.imread('content/me.jpg')
content_image = nst.reshape_and_normalize_image(content_image)


In [0]:
style_image = cv2.imread("style/scream.jpg")
style_image = nst.reshape_and_normalize_image(style_image)

In [0]:
generated_image = nst.generate_noise_image(content_image)

In [0]:
model = nst.load_vgg_model("pretrained_model/imagenet-vgg-verydeep-19.mat")

In [0]:
sess.run(model['input'].assign(content_image))
out = model['conv4_2']
a_C = sess.run(out)
a_G = out

J_content = content_cost(a_C, a_G)

sess.run(model['input'].assign(style_image))
J_style = full_style_cost(model, LAYER_WEIGHTS)

In [0]:
J = total_cost(J_content, J_style)

In [0]:
lr = 2
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(J)

Eventually, the model

In [0]:
def nst_model(sess, input_img, num_iters = 4201):
  sess.run(tf.global_variables_initializer())
  sess.run(model['input'].assign(input_img))

  for i in range(num_iters):
    #let's use the learning rate decay
    lr = 2 / (1 + i / 1000)
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)
    

    sess.run(train_step)
    generated_image = sess.run(model["input"])

    if(i % 100 == 0):
      Jt, Jc, Js = sess.run([J, J_content, J_style])
      print("Iteration " + str(i) + " :")
      print("total cost = " + str(Jt))
      print("content cost = " + str(Jc))
      print("style cost = " + str(Js))
            
      nst.save_image("output/" + str(i) + ".jpg", generated_image)

In [0]:
nst_model(sess, generated_image)

Iteration 0 :
total cost = 210223.84
content cost = 8356.866
style cost = 1055.4598
Iteration 100 :
total cost = 42628.223
content cost = 3303.9243
style cost = 79.908165
Iteration 200 :
total cost = 34530.992
content cost = 2537.5667
style cost = 76.29439
Iteration 300 :
total cost = 32269.29
content cost = 2313.7773
style cost = 76.09597
Iteration 400 :
total cost = 30847.926
content cost = 2191.4949
style cost = 74.44146
Iteration 500 :
total cost = 30318.379
content cost = 2148.8337
style cost = 73.58367
